01.04 名稱空間
===
Python 使用了名稱空間 (Namespace) 的概念，但又將過程完全隱藏，而每種程式語言處理的方式都不太一樣，再加上 Python 有可變、不可變物件的規則，這導致在不熟悉 Python 的情況下，此處是一個容易產生隱藏 bug 的地方。

Python 在 **函數** 中，對於名稱的搜尋方式稱為 LEGB-rule，即下列順序：

1. Local
2. Enclosing
3. Global
4. Build-in

如下範例，函數內為 Local，當找不到 a、b 兩個名稱時，會去 Global 搜尋。

In [1]:
a = 1
b = 2

def foo():
    print(a)
    print(b)
    
foo()

1
2


如下範例，因為在 Local 中有 ```b =``` 來建立變數，因此 Python 認為是 b 是 Local 變數，因此 print(b) 引發 UnbondLocalError。

In [2]:
a = 1
b = 2

def foo():
    print(a)
    print(b)
    b = 3
    
foo()

1


UnboundLocalError: local variable 'b' referenced before assignment

同理，如下範例，在 ```a = a + 1``` 中試圖呼叫 a 作運算，但發現 Local 中有用 ```a = ``` 建立變數 (就是目前這一行)，因此被認為是 Local 變數。 

In [3]:
a = 1 

def foo():
    a = a + 1
    print(a)
    
foo()

UnboundLocalError: local variable 'a' referenced before assignment

如下範例，在 Local 沒有 ```a = ```，因此會去 Global 搜索。

In [4]:
a = 1 

def foo():
    a + 1
    print(a)
    
foo()

1


同樣概念，參考下列範例是修改 Global 的可變物件，在 foo() 內的 a 在 Global 被找到；而在 boo() 內的 a 被認為是 Local。

In [5]:
a = [1, 2] 

def foo():
    a.append(1)
    print(a)

def boo():
    a = a.append(1)
    print(a)
    
foo()
boo()

[1, 2, 1]


UnboundLocalError: local variable 'a' referenced before assignment

關於 Enclosing 與 Build-in，如下範例：

In [6]:
outer = 'global'

def func():
    enclosing = 'enclosing'
    def inner():
            inner = 'inner'
            print(inner)           # fetch from (L)ocal scope
            print(enclosing)       # fetch from (E)nclosing scope
            print(outer)           # fetch from (G)lobal scope
            print(any)             # fetch from (B)uilt-ins
    inner()

func()

inner
enclosing
global
<built-in function any>


---
## 類別
---
類別在 Python 中代表獨立的 Namespace，其存在位置即為 Global，概念上可以用 c++ 的概念將最根部的 Namespace 用 ```::``` 來表示，則在 Global 的 x 變數則可表示為 ```::x```，在 Class A 空間底下的 x 變數則為 ```::A::x```，此處只是概念的說明，實際在 Python 使用不會看到 ```::``` 符號，在使用上也是有要注意的地方。

如下範例，在 class A 第一次呼叫 x 時以及運算時，會找到 ```::x```，但是在執行 ```x =``` 時，隱含著 ```A.x =``` (即 ```::A::x =```) 的概念。

In [7]:
x = 'outer'

class A:
    print('first x in class:', x)
    x = x + 'inside'  # use the value of global `x` to create a new attribute `A.x`
    print('second x in class:', x)          # prints `A.x`

print('x in global:', x)

first x in class: outer
second x in class: outerinside
x in global: outer


在 class 中使用 ```global``` 關鍵字，如下範例，在 class A 中的 ```x =``` 將不再隱含 ```A.x = ``` 的概念。

In [8]:
x = 'outer'

class A:
    global x
    x += 'inner' #now x is not a class attribute, you just modified the global x
    print('x in class:', x)

print('x in global:', x)
print(A.x)

x in class: outerinner
x in global: outerinner


AttributeError: type object 'A' has no attribute 'x'

In [9]:
x = 'outer'

class A:
    print('first x in class:', x)
    x = x + 'inside'  # use the value of global `x` to create a new attribute `A.x`
    print('second x in class:', x) # prints `A.x`

print('x in global:', x)

first x in class: outer
second x in class: outerinside
x in global: outer


一個令人討厭的情形，是直接在 class 內使用 generator 來作 List Comprehension，如下範例，其中 ```(x+i for i in range(5))``` 是產生一個 generator 物件，然後使用 ```[ ]``` 產生 list，這代表在 generator 內的 ```_ge``` 函數會遵循 LEGB-rule 去 Enclosing 找 x (即 ```::x```)，因此會引發 NameError。

若是在 class B 內使用 ```(B.x+i for i in range(5))``` 也會引發錯誤，因為 Python 並沒有提供相對 Namespace 呼叫的方式，因此在 class B 中呼叫 ```B.x``` 也會引發 NameError。

In [10]:
class B:
    x = 3
    y = [x+i for i in range(5)]
    
print(B.y)

TypeError: Can't convert 'int' object to str implicitly

In [11]:
x = 3

class C:
    y = [x+i for i in range(5)]
    
print(C.y)

[3, 4, 5, 6, 7]
